In [12]:
from math import pi

import pandas as pd
import numpy as np

from bokeh.plotting import figure
from bokeh.io import show, save, output_notebook, output_file, export_png
from bokeh.models import  ColumnDataSource, LabelSet, PrintfTickFormatter, FuncTickFormatter
from bokeh.palettes import Category20c
from bokeh.transform import cumsum
from bokeh.core.properties import value
from bokeh.transform import dodge

output_notebook()

data = {'Type':['Emergency', 'Normal'],
 'Canceled': np.random.randint(0, 100, 2),
 'Closed':np.random.randint(0, 100, 2),
 'Implement':np.random.randint(0, 100, 2),
 'Review':np.random.randint(0, 100, 2),
 'Scheduled': np.random.randint(0, 100, 2),
 'Total':np.random.randint(100, 500, 2)}

df = pd.DataFrame(data)

data = df[['Type','Total']]
type = data['Type'].tolist()
state = data.columns.tolist()[1:-1]
total = data['Total'].tolist()

data['angle'] = data['Total'] / data['Total'].sum() * 2*pi
data['color'] = ["#e6550d", "#6baed6"]

value = data['Total'].tolist()

data['cum_angle'] = [ (sum(value[0: i+1]) - (item/2)) / sum(value)*2*pi for i, item in enumerate(value) ]

data['percent'] = data['Total'].apply(lambda x: (x / data['Total'].sum()) * 100 )

data['label'] = [ "{:.0f}%".format(val) for val in data['percent'] ]

data['cos'] = np.cos(data['cum_angle']) * 0.25
data['sin'] = np.sin(data['cum_angle']) * 0.25

source = ColumnDataSource(data)

p = figure(height=450, width=650, toolbar_location=None, title="Total CHG CAB Approved in Mar 2023",
           x_range=(-0.5, 1))

p.wedge(x=0.2, y=0, radius=0.47,
          start_angle=cumsum('angle', include_zero=True), 
          end_angle=cumsum('angle'),
          line_color='white',
          fill_color='color',
          legend_field='Type',
          source=source)

labels = LabelSet(x='cos', y='sin', x_offset=100, y_offset=-0.2, level='glyph',
                  text='label', text_font_size='19px', text_color='black', text_align='center', source=source)

p.add_layout(labels)

title = p.title
title.text_font_size = '24px'
title.align = 'center'

leg = p.legend
leg.label_text_font_size = '16px'

x = p.axis
x.axis_label = None
x.visible=False

zgri = p.grid
zgri.grid_line_color = None


# output_file("chg_pie.html")
# save(p)

export_png(p, filename="chg_pie.png")

Loading BokehJS ...

/tmp/ipykernel_8639/1139243837.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['angle'] = data['Total'] / data['Total'].sum() * 2*pi
/tmp/ipykernel_8639/1139243837.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['color'] = ["#e6550d", "#6baed6"]
/tmp/ipykernel_8639/1139243837.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

RuntimeError: To use bokeh.io image export functions you need selenium ('conda install selenium' or 'pip install selenium')

In [5]:
import html5lib
pd.read_html('chg_pie.html', header=0)

ModuleNotFoundError: No module named 'html5lib'

In [26]:
data

,Type,Total,angle,color,cum_angle,percent,label,cos,sin
0,Emergency,305,2.479135,#e6550d,1.239568,39.456662,39%,0.081301,0.236411
1,Normal,468,3.804050,#6baed6,4.381160,60.543338,61%,-0.081301,-0.236411


In [1]:
from math import pi

import pandas as pd
import numpy as np

from bokeh.plotting import figure
from bokeh.io import show, output_notebook, output_file
from bokeh.models import  ColumnDataSource, LabelSet, PrintfTickFormatter, FuncTickFormatter
from bokeh.palettes import Category20c
from bokeh.transform import cumsum
from bokeh.core.properties import value
from bokeh.transform import dodge

output_notebook()





towerleads = ['Adil', 'Dmitriy', 'Gabriele', 'Joe', 'John', 'Joseph', 'Katsunori', 'Leo', 'Mark', 'Mattieu', 'Michael', 'Nitin', 'Sebastien', 'Shankar', 'Shishir', 'Stephen']

dates = ['Oct 2022', 'Nov 2022', 'Dec 2022', 'Jan 2023', 'Feb 2023', 'Mar 2023']
data = { date:np.random.randint(10, 100, len(towerleads)) for date in dates}

data = pd.DataFrame(data, index=towerleads).reset_index().rename(columns={'index':'Tower Leads'})

source = ColumnDataSource(data=data)

p = figure(x_range=(0, 500), y_range=towerleads, plot_height=550, plot_width=1000, title="CHG by Tower Leads - CAB Approved 6M Trends",
           tools='hover', tooltips='$name: @$name')

p.hbar_stack(dates, y='Tower Leads', height=0.9, color=Category20c[len(dates)], source=source, legend_label=["%s" % x for x in dates])

"""
numpy.flatten() is a method in NumPy that returns a copy of an array in one-dimensional form (i.e., a flattened array). It collapses all the dimensions of the input array into a single axis, which is returned in the order of the last axis.
    x - cumsum by row and then subtract half the bar to center the label

    y - populate the list of y values with the midpoint of each bar

    z - convert the data to a string and replace 0% with an empty string
"""

x = list((data[dates].cumsum(axis=1) - data[dates]/2).to_numpy().flatten()) 
y = list(np.array([i+0.5 for i in range(len(towerleads))]).repeat(len(dates)))*2
labels = list(data[dates].abs().astype(str).replace('^0$','', regex=True).to_numpy().flatten())

labels = {'x': x, 'y':y, 'labels':labels}
labelset = LabelSet(x='x', y='y', text='labels', text_font_size='11px', text_color= 'white',source=ColumnDataSource(labels), text_align='center')

p.add_layout(labelset)

x = p.xaxis
y = p.yaxis
z = p.axis
x.axis_label = "# of CAB Approved CHG"
x.axis_label_text_font = "helvetica"
x.axis_label_text_font_size = "15px"
x.axis_label_text_font_style = "bold"
x.major_label_text_font_size = "15px"
x.formatter = FuncTickFormatter(code="return Math.abs(tick)")

y.axis_label = "Tower Leads"
y.axis_label_text_font = "helvetica"
y.axis_label_text_font_size = "16px"
y.major_label_text_font_size = "14px"
y.major_label_text_font_style = "bold"

# z.axis_label_text_font_style = "bold"

ygri = p.ygrid
ygri.visible = False 

title = p.title
title.text_font_size = '24px'
title.align = 'center'

leg = p.legend
leg.title = 'Timeline'
leg.label_text_alpha = 0.75

show(p)

Loading BokehJS ...